In [1]:
## Importing pakcages
import pandas as pd
import numpy as np

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN
from keras.callbacks import EarlyStopping

from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

import itertools

import warnings
warnings.filterwarnings("ignore")

tf.random.set_seed(111)

2022-03-03 18:52:25.301191: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/spack/bell/apps/gcc/9.3.0-gcc-4.8.5-z2ug7z7/lib64:/apps/spack/bell/apps/gcc/9.3.0-gcc-4.8.5-z2ug7z7/lib:/apps/spack/bell/apps/zlib/1.2.11-gcc-9.3.0-7x3gabj/lib:/apps/spack/bell/apps/mpc/1.1.0-gcc-4.8.5-wwn2jff/lib:/apps/spack/bell/apps/mpfr/3.1.6-gcc-4.8.5-xyisskj/lib:/apps/spack/bell/apps/gmp/6.1.2-gcc-4.8.5-6bsovvk/lib
2022-03-03 18:52:25.301246: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [1]:
# Data Preprocessing --- after extracting PID relevant data into a csv format

In [193]:
df = pd.read_csv('pid519494.0.csv')

In [187]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494834 entries, 0 to 494833
Columns: 165 entries, UT_ID to NYEVE
dtypes: float64(18), int64(141), object(6)
memory usage: 622.9+ MB


In [195]:
df = df[df['ATL_T_SNOW_QT'] != -999.99]

In [196]:
df = df[df['DAY_BEF_SNOW_TEMP_VAL'] != -999.99]

In [197]:
df = df[df['DAY_AFT_SNOW_TEMP_VAL'] != -999.99]

In [198]:
df.to_csv('cleaned_data_holiday.csv', index=False)

In [199]:
# Make a new holiday column with excel functions on excel (for easier row-wise operations)

In [205]:
# Reading back new file
df = pd.read_csv('cleaned_data_holiday.csv')

In [206]:
# Obtaining column index to slice off columns
df.columns.get_loc('Holiday')

44

In [207]:
# Removing holidays one-hot encoded columns from EDA
df = df.iloc[:,0:45]

In [208]:
# Dropping transformed columns for EDA
df.drop(['NTL_LOG_MKT_BSKT_UT_QT', 'NTL_LOG_BSE_PR_AM', 'NTL_LOG_BSE_DCT_PR_AM', 'NTL_LOG_BSE_PROMO_DCT_PR_AM',
         'pr_drop_flg', "adv_mid_wk_flg", 'adv_super_evnt_flg', 'adv_dgtl_circ_flg'],
        axis=1, inplace=True)

In [215]:
df['DAY_DT'] = pd.to_datetime(df['DAY_DT'])
df['p_sold_first_dt'] = pd.to_datetime(df['p_sold_first_dt'])
df['p_sold_last_dt'] = pd.to_datetime(df['p_sold_last_dt'])
df['WK_END_DT'] = pd.to_datetime(df['WK_END_DT'])

In [211]:
df['CLDR_DAY_OF_WK_ID'].nunique()

7

In [213]:
df.astype('object').describe().transpose()

,count,unique,top,freq
UT_ID,474199,254,30,2237
P_ID,474199,1,519494,474199
DAY_DT,474199,2049,2021-08-06 00:00:00,254
MJR_P_CLS_ID,474199,1,b'L2-013115',474199
p_sold_first_dt,474199,35,2017-10-10 00:00:00,229511
p_sold_last_dt,474199,1157,2018-01-13 00:00:00,163960
P_SELL_DUR,474199,1521,1273,147284
MKT_BSKT_UT_QT,474199,29601,0,3521
P_BSE_PR_AM,474182,406,5.99,133864
P_PROMO_PR_AM,474199,459,4.42,37150


In [214]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
UT_ID,474199.0,174.725965,81.895322,19.00,118.00,178.00,242.00,324.00
P_ID,474199.0,519494.000000,0.000000,519494.00,519494.00,519494.00,519494.00,519494.00
P_SELL_DUR,474199.0,1066.985399,314.237482,0.00,843.00,1273.00,1273.00,1533.00
MKT_BSKT_UT_QT,474199.0,59.379615,58.266553,0.00,22.89,41.64,75.35,1215.04
P_BSE_PR_AM,474182.0,6.027269,0.614492,0.18,5.74,5.99,6.24,25.11
P_PROMO_PR_AM,474199.0,4.472060,0.526542,2.24,4.29,4.38,4.45,8.99
ATL_P_SOLD_AM,474199.0,4.799041,0.948898,0.18,3.99,4.65,5.47,8.99
adv_circ_flg,474199.0,0.464725,0.498755,0.00,0.00,0.00,1.00,1.00
adv_mid_wk_flg,474199.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00
adv_super_evnt_flg,474199.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00


In [223]:
# Sales price for actual product sold
df['sales_price_alt'] = df['MKT_BSKT_UT_QT']*df['ATL_P_SOLD_AM']

In [228]:
df.to_csv('cleaned_data_v3.csv', index=False)

In [2]:
# Model Building

In [2]:
data=pd.read_csv("/home/sharm526/Team_2_NMIMS/Final/cleaned_data_v3.csv")

In [3]:
def convert2matrix(data_arr, look_back):
    X, Y =[], []
    for i in range(len(data_arr)-look_back):
        d=i+look_back  
        X.append(data_arr[i:d,0])
        Y.append(data_arr[d,0])
    return np.array(X), np.array(Y)

In [4]:
# DNN

In [5]:
def model_dnn(look_back):
    model=Sequential()
    model.add(Dense(units=32, input_dim=look_back, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
    return model

In [6]:
df_1=pd.DataFrame(columns=['UT_ID','WAPE'])
temp=list(data['UT_ID'].unique())
for x in temp:
    data1 = data[data["UT_ID"] == x][["DAY_DT", "MKT_BSKT_UT_QT"]].sort_values("DAY_DT").reset_index(drop = True)
    data1['Year']=data1['DAY_DT'].apply(lambda x:x[:4])
    temp_rough_temp=sorted(list(set(data1['Year'].to_list())))
    if(len(temp_rough_temp)==1):
        df_1=df_1.append({'UT_ID':x,'WAPE':'X'},ignore_index=True)
        continue
    train=data1[data1['DAY_DT']<='2021-05-31'].copy()
    test=data1[data1['DAY_DT']>'2021-05-31'].copy()
    train.set_index("DAY_DT", inplace = True)
    test.set_index("DAY_DT", inplace = True)
    train.drop('Year',axis=1,inplace=True)
    test.drop('Year',axis=1,inplace=True)
    train,test=train.values[:],test.values[:]
    trainX, trainY = convert2matrix(train, len(test)//2)
    testX, testY = convert2matrix(test, len(test)//2)
    model=model_dnn(len(test)//2)
    history=model.fit(trainX,trainY, epochs=100, batch_size=30, verbose=0, validation_data=(testX,testY),
                  callbacks=[tf.keras.callbacks.EarlyStopping(monitor= "val_loss" , patience=10)],shuffle=False)
    train_predict = model.predict(trainX)
    test_predict = model.predict(testX)
    df_1=df_1.append({'UT_ID':x,'WAPE':round(np.abs(testY - test_predict.reshape(testY.shape)).sum()/np.abs(testY).sum(),2)},ignore_index=True)
df_1.to_csv('/home/sharm526/Team_2_NMIMS/Final/DNN.csv', index=False)

2022-03-03 18:53:01.741960: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/spack/bell/apps/gcc/9.3.0-gcc-4.8.5-z2ug7z7/lib64:/apps/spack/bell/apps/gcc/9.3.0-gcc-4.8.5-z2ug7z7/lib:/apps/spack/bell/apps/zlib/1.2.11-gcc-9.3.0-7x3gabj/lib:/apps/spack/bell/apps/mpc/1.1.0-gcc-4.8.5-wwn2jff/lib:/apps/spack/bell/apps/mpfr/3.1.6-gcc-4.8.5-xyisskj/lib:/apps/spack/bell/apps/gmp/6.1.2-gcc-4.8.5-6bsovvk/lib
2022-03-03 18:53:01.741993: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-03 18:53:01.742013: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bell-a009.rcac.purdue.edu): /proc/driver/nvidia/version does not exist
2022-03-03 18:53:01.742214: I tensorflow/core/platform/cpu_feature_guard.cc:151] This Tensor

In [13]:
print('Min Wape: ',str(min(df_1[df_1['WAPE']!='X']['WAPE'].to_list())))
print('Max Wape: ',str(max(df_1[df_1['WAPE']!='X']['WAPE'].to_list())))
print('Sum Wape: ',str(sum(df_1[df_1['WAPE']!='X']['WAPE'].to_list())))
print('Avg Wape: ',str(np.mean(df_1[df_1['WAPE']!='X']['WAPE'].to_list())))

Min Wape:  0.2
Max Wape:  0.56
Sum Wape:  85.67000000000006
Avg Wape:  0.3413147410358566


In [14]:
# RNN

In [15]:
def model_rnn(look_back):
    model=Sequential()
    model.add(SimpleRNN(units=32, input_shape=(1,look_back), activation="relu"))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
    return model

In [16]:
df_2=pd.DataFrame(columns=['UT_ID','WAPE'])
temp=list(data['UT_ID'].unique())
for x in temp:
    data2 = data[data["UT_ID"] == x][["DAY_DT", "MKT_BSKT_UT_QT"]].sort_values("DAY_DT").reset_index(drop = True)
    data2['Year']=data2['DAY_DT'].apply(lambda x:x[:4])
    temp_rough_temp=sorted(list(set(data2['Year'].to_list())))
    if(len(temp_rough_temp)==1):
        df_2=df_2.append({'UT_ID':x,'WAPE':'X'},ignore_index=True)
        continue
    train=data2[data2['DAY_DT']<='2021-05-31'].copy()
    test=data2[data2['DAY_DT']>'2021-05-31'].copy()
    train.set_index("DAY_DT", inplace = True)
    test.set_index("DAY_DT", inplace = True)
    train.drop('Year',axis=1,inplace=True)
    test.drop('Year',axis=1,inplace=True)
    train,test=train.values[:],test.values[:]
    trainX, trainY = convert2matrix(train, len(test)//2)
    testX, testY = convert2matrix(test, len(test)//2)
    # reshape input to be [samples, window size, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    model=model_rnn(len(test)//2)
    history=model.fit(trainX,trainY, epochs=100, batch_size=30, verbose=0, validation_data=(testX,testY),
                  callbacks=[tf.keras.callbacks.EarlyStopping(monitor= "val_loss" , patience=10)],shuffle=False)
    train_predict = model.predict(trainX)
    test_predict = model.predict(testX)
    df_2=df_2.append({'UT_ID':x,'WAPE':round(np.abs(testY - test_predict.reshape(testY.shape)).sum()/np.abs(testY).sum(),2)},ignore_index=True)
df_2.to_csv('/home/sharm526/Team_2_NMIMS/Final/RNN.csv', index=False)

In [17]:
print('Min Wape: ',str(min(df_2[df_2['WAPE']!='X']['WAPE'].to_list())))
print('Max Wape: ',str(max(df_2[df_2['WAPE']!='X']['WAPE'].to_list())))
print('Sum Wape: ',str(sum(df_2[df_2['WAPE']!='X']['WAPE'].to_list())))
print('Avg Wape: ',str(np.mean(df_2[df_2['WAPE']!='X']['WAPE'].to_list())))

Min Wape:  0.22
Max Wape:  0.59
Sum Wape:  87.44000000000004
Avg Wape:  0.3483665338645418


In [18]:
# Random Forest Regressor

In [19]:
rfc = RandomForestRegressor(n_estimators = 100, random_state = 0)
df_3=pd.DataFrame(columns=['UT_ID','WAPE'])
temp=list(data['UT_ID'].unique())
for x in temp:
    data3 = data[data["UT_ID"] == x][["DAY_DT", "MKT_BSKT_UT_QT"]].sort_values("DAY_DT").reset_index(drop = True)
    data3['Year']=data3['DAY_DT'].apply(lambda x:x[:4])
    temp_rough_temp=sorted(list(set(data3['Year'].to_list())))
    if(len(temp_rough_temp)==1):
        df_3=df_3.append({'UT_ID':x,'WAPE':'X'},ignore_index=True)
        continue
    train=data3[data3['DAY_DT']<='2021-05-31'].copy()
    test=data3[data3['DAY_DT']>'2021-05-31'].copy()
    train.set_index("DAY_DT", inplace = True)
    test.set_index("DAY_DT", inplace = True)
    train.drop('Year',axis=1,inplace=True)
    test.drop('Year',axis=1,inplace=True)
    train,test=train.values[:],test.values[:]
    trainX, trainY = convert2matrix(train, len(test)//2)
    testX, testY = convert2matrix(test, len(test)//2)
    rfc.fit(trainX, trainY)
    test_predict = rfc.predict(testX)
    df_3=df_3.append({'UT_ID':x,'WAPE':round(np.abs(testY - test_predict.reshape(testY.shape)).sum()/np.abs(testY).sum(),2)},ignore_index=True)
df_3.to_csv('/home/sharm526/Team_2_NMIMS/Final/RF.csv', index=False)

In [20]:
print('Min Wape: ',str(min(df_3[df_3['WAPE']!='X']['WAPE'].to_list())))
print('Max Wape: ',str(max(df_3[df_3['WAPE']!='X']['WAPE'].to_list())))
print('Sum Wape: ',str(sum(df_3[df_3['WAPE']!='X']['WAPE'].to_list())))
print('Avg Wape: ',str(np.mean(df_3[df_3['WAPE']!='X']['WAPE'].to_list())))

Min Wape:  0.2
Max Wape:  0.59
Sum Wape:  84.51999999999995
Avg Wape:  0.3367330677290837


In [1]:
# XGBoost

In [23]:
def model_xgb(temp):
    xg_reg=xgb.XGBRegressor(learning_rate = temp[0][0], max_depth = temp[0][1], n_estimators = temp[1])
    xg_reg.fit(trainX,trainY)
    preds = xg_reg.predict(testX)
    wape=round(np.abs(testY - preds).sum()/np.abs(testY).sum(),2)
    return wape

def param():
    learning_rate=[0.01,0.001,0.0001]
    max_depth=[3,4,5]
    n_estimators=[50,100,200,500]

    temp_list=[list(zip(each_permutation, max_depth)) for each_permutation in itertools.permutations(learning_rate, len(max_depth))]
    temp_list=list(set([x for tup in temp_list for x in tup ]))
    param_grid=[list(zip(each_permutation, n_estimators)) for each_permutation in itertools.permutations(temp_list, len(n_estimators))]
    param_grid=list(set([x for tup in param_grid for x in tup ]))
    return param_grid

In [25]:
df_4=pd.DataFrame(columns=['UT_ID','WAPE','Learning Rate','Max Depth','N_Estimators','Size'])
temp=list(data['UT_ID'].unique())
for x in temp:
    mod=[]
    wap=[]
    data4 = data[data["UT_ID"] == x][["DAY_DT", "MKT_BSKT_UT_QT"]].sort_values("DAY_DT").reset_index(drop = True)
    data4['Year']=data4['DAY_DT'].apply(lambda x:x[:4])
    temp_rough_temp=sorted(list(set(data4['Year'].to_list())))
    if(len(temp_rough_temp)==1):
        df_4=df_4.append({'UT_ID':x,'WAPE':'X'},ignore_index=True)
        continue
    train=data4[data4['DAY_DT']<='2021-05-31'].copy()
    test=data4[data4['DAY_DT']>'2021-05-31'].copy()
    train.set_index("DAY_DT", inplace = True)
    test.set_index("DAY_DT", inplace = True)
    train.drop('Year',axis=1,inplace=True)
    test.drop('Year',axis=1,inplace=True)
    train,test=train.values[:],test.values[:]
    trainX, trainY = convert2matrix(train, len(test)//2)
    testX, testY = convert2matrix(test, len(test)//2)
    param_grid=param()
    for par in param_grid:
        mod.append(par)
        wap.append(model_xgb(par))
    df_4=df_4.append({'UT_ID':x,'WAPE':min(wap),'Learning Rate':mod[wap.index(min(wap))][0][0],'Max Depth':mod[wap.index(min(wap))][0][1],'N_Estimators':mod[wap.index(min(wap))][1],'Size':len(test)//2},ignore_index=True)
df_4.to_csv('/home/sharm526/Team_2_NMIMS/Final/xgboost.csv',index=False)

In [26]:
print('Min Wape: ',str(min(df_4[df_4['WAPE']!='X']['WAPE'].to_list())))
print('Max Wape: ',str(max(df_4[df_4['WAPE']!='X']['WAPE'].to_list())))
print('Sum Wape: ',str(sum(df_4[df_4['WAPE']!='X']['WAPE'].to_list())))
print('Avg Wape: ',str(np.mean(df_4[df_4['WAPE']!='X']['WAPE'].to_list())))

Min Wape:  0.21
Max Wape:  0.49
Sum Wape:  80.36999999999998
Avg Wape:  0.32019920318725104


In [22]:
# Future Predictions

In [37]:
def convert2matrix_new(data_arr, look_back):
    X, Y =[], []
    for i in range(len(data_arr)-look_back):
        d=i+look_back  
        X.append(data_arr[i:d])
        Y.append(data_arr[d])
    return np.array(X), np.array(Y)

In [70]:
df_mod=pd.read_csv('/home/sharm526/Team_2_NMIMS/Final/xgboost.csv')
temp=list(data['UT_ID'].unique())
col=['Date']
col.extend(list(map(str,temp)))
df_preds=pd.DataFrame(columns=col)
df_preds['Date']=pd.date_range(start='2021-08-29',end='2021-09-30').to_pydatetime().tolist()
count=0
for x in temp:
    data_train = data[data["UT_ID"] == x][["DAY_DT", "MKT_BSKT_UT_QT"]].sort_values("DAY_DT").reset_index(drop = True)
    data_full=data_train['MKT_BSKT_UT_QT'].values[:]
    if(df_mod.loc[count,'WAPE']=='X'):
        df_preds[str(x)]='X'
        count+=1
        continue
    trainX_full, trainY_full = convert2matrix_new(data_full, int(df_mod.loc[count,'Size']))
    xg_reg=xgb.XGBRegressor(learning_rate = df_mod.loc[count,'Learning Rate'], max_depth = int(df_mod.loc[count,'Max Depth']), n_estimators = int(df_mod.loc[count,'N_Estimators']))
    xg_reg.fit(trainX_full,trainY_full)
    test=np.array([data_full[j] for j in range(-int(df_mod.loc[count,'Size']),0)])
    for j in range(df_preds.shape[0]):
        pred= xg_reg.predict(test.reshape(1,len(test)))
        df_preds.loc[j,str(x)]=pred
        test=np.append(test[1:],pred)
    count+=1
df_preds.to_csv('/home/sharm526/Team_2_NMIMS/Final/Final Predictions.csv',index=False)